In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.20.0


In [3]:
True_News = pd.read_csv("True.csv")
True_News['target'] = 1
Fake_News = pd.read_csv("Fake.csv")
Fake_News['target'] = 0

In [4]:
True_News.drop('date', axis =1, inplace =True)
Fake_News.drop('date', axis =1, inplace = True)
df = pd.concat([True_News, Fake_News], ignore_index=True )
df.head()

,title,text,subject,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,1


In [5]:
df = df.sample(frac = 1.0, ignore_index=True)
X = df.drop('target', axis = 1) 
y = df.target

In [6]:
X.shape ,y.shape

((44898, 3), (44898,))

In [7]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense

In [8]:
voc_size = 5000

In [9]:
news = X.copy()
news['title'][0]

'EU braces for Brexit talks collapse as May falters'

In [10]:
import nltk
import re
from nltk.corpus import stopwords

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(news)):
    review = re.sub('[^a-zA-Z]', ' ', news['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [13]:
one_hot_repr = [one_hot(words, voc_size) for words in corpus]
one_hot_repr

[[4689, 2569, 3935, 1358, 4052, 4264, 1287],
 [1539, 3043, 575, 2122, 304, 1196, 1924, 1196],
 [932, 2341, 2916, 1871, 890, 4954, 3098, 2058, 1276, 4961, 731, 2832],
 [243, 297, 3751, 3779, 1020, 633, 4787, 1061, 1210, 1116, 387, 1064, 1072],
 [1490, 1038, 1866, 111, 3914, 3423, 2558, 4993],
 [422, 4025, 2761, 783, 2228, 3182, 1358, 3200],
 [1977, 2280, 1622, 3640],
 [4091, 2020, 2919, 3596, 3213, 2582, 3430, 2268, 826, 4558, 273],
 [422, 4217, 4021, 3833, 2973, 3837, 992, 3186],
 [4262,
  1650,
  3487,
  3395,
  4525,
  3768,
  3770,
  613,
  3914,
  1114,
  3475,
  3043,
  3468,
  2272,
  2761],
 [2846, 3110, 3963, 2633, 4899, 771, 3875, 481, 1571, 3037],
 [1393, 4217, 809, 2139, 3779, 2689, 2185, 519, 1140, 4642, 3830, 3470, 1072],
 [2362, 3466, 4655, 2028, 4072, 2298],
 [2366, 292, 4773, 945, 3489, 1522, 214, 3596],
 [4965, 735, 1260, 352, 2182, 891, 2704, 3370, 3256],
 [2989, 1919, 4720, 3206, 1369, 875],
 [1116, 4264, 1320, 1490, 2110, 3110, 135, 2912, 868],
 [243, 1022, 1353, 42

In [14]:
corpus

['eu brace brexit talk collaps may falter',
 'would defend famili violent rioter think happen think',
 'break megyn kelli gave ultim f k fox news could destroy network',
 'bodi languag expert give clear exampl susan rice lie trump surveil interview video',
 'qatar flex financi muscl billion euro french deal',
 'hous tax posit begin emerg ahead talk senat',
 'bannon loss keep come',
 'whoa dc diva blow traffic law suv shop fanci shoe store',
 'hous republican tri hand america elect koch brother',
 'updat transgend target boycott reach boil point lose billion day ceo defend danger bathroom posit',
 'iraqi forc launch offens recaptur last town islam state control',
 'tucker carlson brutal ms gang obama undocu children far greater threat isi video',
 'mexico report flag shortcom fight corrupt',
 'china overwhelm advantag bring taiwan heel offici say',
 'kenyan opposit leader odinga withdrew vote run call protest',
 'brazil meirel joke vice presidenti candidaci',
 'trump may halt insur paym

In [15]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4052 4264 1287]
 [   0    0    0 ... 1196 1924 1196]
 [   0    0    0 ... 4961  731 2832]
 ...
 [   0    0    0 ... 3872 2594 1073]
 [   0    0    0 ... 1933 1120 3825]
 [   0    0    0 ... 2847  768  187]]


In [16]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 4689, 2569, 3935, 1358, 4052, 4264, 1287], dtype=int32)

In [17]:
from tensorflow.keras.layers import Dropout

embedding_vector_features = 40 
model = Sequential()
model.add(Embedding(
    input_dim=voc_size,
    output_dim=embedding_vector_features,
    input_shape=(sent_length,)
))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

C:\Users\Gaurav\Desktop\ML Projects\KrishNaik\env\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20, 40)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 200)                 │         112,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [19]:
X_final.shape,y_final.shape

((44898, 20), (44898,))

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9011 - loss: 0.2337 - val_accuracy: 0.9324 - val_loss: 0.1661
Epoch 2/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9496 - loss: 0.1329 - val_accuracy: 0.9374 - val_loss: 0.1561
Epoch 3/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9594 - loss: 0.1083 - val_accuracy: 0.9401 - val_loss: 0.1490
Epoch 4/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9661 - loss: 0.0925 - val_accuracy: 0.9372 - val_loss: 0.1716
Epoch 5/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9729 - loss: 0.0771 - val_accuracy: 0.9427 - val_loss: 0.1709
Epoch 6/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9753 - loss: 0.0697 - val_accuracy: 0.9409 - val_loss: 0.1689
Epoch 7/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9781 - loss: 0.0619 - val_accuracy: 0.9400 - val_loss: 0.1910
Epoch 8/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.9807 - loss: 0.0538 - val_accu

In [22]:
y_pred=model.predict(X_test)
y_pred = (y_pred > 0.5)

464/464 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[7328,  495],
       [ 397, 6597]])

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9397988796652493

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94      7823
           1       0.93      0.94      0.94      6994

    accuracy                           0.94     14817
   macro avg       0.94      0.94      0.94     14817
weighted avg       0.94      0.94      0.94     14817

